In [16]:
import nshutils as nu
import rich

nu.pretty()

In [4]:
from __future__ import annotations

import datasets

datasets.disable_caching()

alex_og = datasets.load_dataset(
    "nimashoghi/alexandria-pbe-geo-opt-paths-raw", streaming=True, split="train"
)
alex_og

Resolving data files:   0%|          | 0/450 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/450 [00:00<?, ?it/s]

IterableDataset({
    features: ['material_id', 'numbers', 'positions', 'cell', 'pbc', 'num_atoms', 'composition', 'energy', 'forces', 'stress'],
    n_shards: 450
})

In [5]:
salex = datasets.load_dataset("nimashoghi/salex", split="all")
salex

Dataset({
    features: ['pos', 'cell', 'atomic_numbers', 'natoms', 'tags', 'sid', 'energy', 'forces', 'stress', 'fixed', 'pbc', 'fid'],
    num_rows: 11000983
})

In [6]:
salex_mapped = salex.map(
    lambda sid: {"material_id": sid.split("_", 1)[0]}, input_columns=["sid"]
)
salex_mapped

Map:   0%|          | 0/11000983 [00:00<?, ? examples/s]

Dataset({
    features: ['pos', 'cell', 'atomic_numbers', 'natoms', 'tags', 'sid', 'energy', 'forces', 'stress', 'fixed', 'pbc', 'fid', 'material_id'],
    num_rows: 11000983
})

In [8]:
idset = set(salex_mapped["material_id"])
print(len(idset))

3398156


In [18]:
idfound = None
for row in alex_og:
    if row["material_id"] in idset:
        idfound = row["material_id"]
        break

print(idfound)

# Find index of idfound in salex_mapped
print(salex_idx := salex_mapped["material_id"].index(idfound))

agm003285455
7668017
7668017


In [24]:
from typing import Any

import numpy as np


def convert(d: dict[str, Any]):
    return {
        k: np.array(v) if isinstance(v, (list, float, int)) else v
        for k, v in sorted(d.items(), key=lambda x: x[0])
    }


rich.print(row_og := convert(row))
rich.print(row_salex := convert(salex_mapped[salex_idx + 1]))

{
    'cell': array[3, 3] n=9 x∈[-4.172, 7.776] μ=1.618 σ=3.855 [[5.111, 0.257, 0.032], [-0.329, 7.228, -0.608], 
[-0.737, -4.172, 7.776]],
    'composition': array[119] i64 x∈[0, 13] μ=0.126 σ=1.199,
    'energy': array -38.897,
    'forces': array[15, 3] n=45 x∈[-0.006, 0.006] μ=9.637e-21 σ=0.002,
    'material_id': 'agm003285455',
    'num_atoms': array i64 15,
    'numbers': array[15] i64 x∈[47, 48] μ=47.133 σ=0.340,
    'pbc': array[3] bool x∈[True, True] μ=1.000 σ=0. [True, True, True],
    'positions': array[15, 3] n=45 x∈[-1.950, 7.390] μ=2.265 σ=2.106,
    'stress': array[3, 3] n=9 x∈[-6.098, 0.882] μ=-1.999 σ=2.283 [[-6.098, 0.882, -1.122], [0.882, -4.735, -1.376],
[-1.122, -1.376, -3.923]]
}

{
    'atomic_numbers': array[15] x∈[47.000, 48.000] μ=47.133 σ=0.340,
    'cell': array[1, 3, 3] n=9 x∈[-4.172, 7.776] μ=1.618 σ=3.855 [[[5.111, 0.257, 0.032], [-0.329, 7.228, -0.608], 
[-0.737, -4.172, 7.776]]],
    'energy': array -38.897,
    'fid': array[1] i64 [0],
    'fixed': array[15] i64 all_zeros,
    'forces': array[15, 3] n=45 x∈[-0.006, 0.006] μ=0. σ=0.002,
    'material_id': 'agm003285455',
    'natoms': array i64 15,
    'pbc': array[3] bool x∈[True, True] μ=1.000 σ=0. [True, True, True],
    'pos': array[15, 3] n=45 x∈[-1.950, 7.390] μ=2.265 σ=2.106,
    'sid': 'agm003285455_2_11',
    'stress': array[1, 3, 3] n=9 x∈[-0.001, 0.004] μ=0.001 σ=0.001 [[[0.004, -0.001, 0.001], [-0.001, 0.003, 
0.001], [0.001, 0.001, 0.002]]],
    'tags': array[15] x∈[1.000, 1.000] μ=1.000 σ=0.
}

In [35]:
nu.display(
    {
        "og": row_og["energy"],
        "salex": row_salex["energy"],
        "diff": row_og["energy"] - row_salex["energy"],
    }
)
nu.display(
    {
        "og": row_og["forces"].T,
        "salex": row_salex["forces"].T,
        "diff": (row_og["forces"] - row_salex["forces"]).T,
    }
)
nu.display(
    {
        "og": row_og["positions"].T,
        "salex": row_salex["pos"].T,
        "diff": (row_og["positions"] - row_salex["pos"]).T,
    }
)
nu.display(
    {
        "og": row_og["stress"],
        "salex": row_salex["stress"],
        "diff": row_og["stress"] - row_salex["stress"],
    }
)

In [41]:
nu.display(
    np.stack(
        [
            row_og["stress"],
            row_salex["stress"][0],
            row_og["stress"] - row_salex["stress"][0],
        ],
        axis=-2,
    )
)

In [52]:
og_stress = row_og["stress"]
salex_stress = row_salex["stress"][0]

nu.display(og_stress / salex_stress)

# https://github.com/ACEsuit/mace/discussions/542#discussioncomment-10263111


SyntaxError: invalid syntax (441601206.py, line 6)

In [53]:
og_stress = row_og["stress"]
salex_stress = row_salex["stress"][0] * -1602.17663

nu.display(
    np.stack(
        [
            og_stress,
            salex_stress,
            og_stress - salex_stress,
        ],
        axis=-2,
    )
)